LSTM 네트워크를 사용해서 텍스트 생성하기 (라이브러리 없이)
-------------------------------------

-----------

**데모**<br>

Eminem 가사 생성을 위해 numpy만 사용해서 구축한 LSTM 네트워크를 트레이닝 시킬것이다. LSTM은 뉴럴 네트워크의 심플한 확장 버전이고, 지난 몇 년간 딥러닝의 많은 성취에 기여했다.<br><br>

*eminem의 lose yourself를 임의로 가져다가 txt 파일로 만들었습니다. 코드는 돌아갈 정도로만 수정했습니다.<br>

----------

**Recurrent Network는 무엇인가?**<br>


Recurrent nets은 데이터의 시퀀스를 배우는데에 유용하다. Input, Hidden state, output<br><br>

<img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-rolled.png' width='200' height='200'><br><br>


input에서 hidden state로 이어진 weight matrix다. weight matrix는 hidden state에서 이전 time step의 hidden state로도 연결되어 있다.<br><br>
<img src='https://iamtrask.github.io/img/basic_recurrence_singleton.png' width='300' height='300'><br><br>


그래서 다음 training iteration의 input뿐 아니라 input + 이전의 hidden state를 전달하는, 시간마다 자기자신과 연결되는 feedforward network로 간주할 수도 있다.<br><br>
<img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png' width='500' height='500'>

------------

<br><br>**Recurrent Network의 문제점**<br><br>

만약 우리가 "The grass is green" 문장의 마지막 단어를 예측하려고 한다면? 가능하다.<br>
<img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png' width='300' height='300'><br><br><br>

만약 우리가 "I am French (2000 단어 뒤에) i speak fluent French" 문장 뒤의 마지막 단어를 예측하려고 한다면? dependencies 길게 기억하는 일이 필요한데, RNN은 이걸 잘 못한다.<br>
<img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png' width='400' height='400'><br><br><br>

이건 "Vanishing Gradient Problem"이라고 하는데, backpropagation 때 Gradient가 exponentially하게 줄어드는 현상이다.<br><br>
<img src='http://slideplayer.com/slide/5251503/16/images/6/Recurrent+Neural+Networks.jpg' width='300' height='300'><br>
<img src='https://cdn-images-1.medium.com/max/1600/1*8JJ6sYleUtvUZR7TOyyFVg.png' width='300' height='300'><br><br><br>

gradient의 magnitude에 영향을 끼치는 두 개의 factor가 있다 - gradient를 지나가는 weight와 activation function이다. (더 정확하게는, 이 녀석들의 도함수-derivatives)<br>
만약 이 factor중 하나가 1보다 작다면, gradient는 사라질수도 있다; 만약 1보다 크다면, 폭발할 수 있다. 해결책은 LSTM 셀을 넣으면 된다!<br><br>

------------

**LSTM 셀 (Long-Short Term Memory Cell)**<br><br>

우리 모델 업데이트를 어떻게 할지 어떤 제약도 없다. 그러니 knowledge를 좀 무질서하게 바꿀수도 있다: <br>
한 프레임에는 미국에 있는 characters, 다음 프레임에는 일본에 있다고 생각되어지는, 초밥을 먹는 characters를 본다. 
<br>그리고 또 다음 프레임은 하이드라섬에 있다고 생각되어지는, 북극곰을 본다.<br><br>

이 혼란의 의미는 정보는 빠르게 사라지고 바뀐다는 말이다. 모델이 long-term memory를 지키기 어렵다는 말.<br>
그러니 네트워크가 배우게 하려면 어떻게 its beliefs를 업데이트 해야하는지를 배우게 하는게 좋다. <br>(Bob이 없는 씬은 Bob-related information을 바꿔선 안되고, Alice씬은 그녀에 대한 세부 정보들을 모으는데에 중점을 둬야한다.)<br><br>

이건 일반적인 RNN 셀을 input, forget, output gate를 써서 대체한다. cell state가 그러하듯.<br><br>
<img src='https://www.researchgate.net/profile/Mohsen_Fayyaz/publication/306377072/figure/fig2/AS:398082849165314@1471921755580/Fig-2-An-example-of-a-basic-LSTM-cell-left-and-a-basic-RNN-cell-right-Figure.ppm' width='400' height='400'><br>
<img src='https://kijungyoon.github.io/assets/images/lstm.png' width='300' height='300'><br><br>


이 gates들은 각자의 weight values의 set을 갖고 있다. 전부 미분가능해야 하는데 그래야 우리가 backprop을 할 수 있다.<br>(우리가 얘네들을 사용해서 gradient를 계산하고 weights를 업데이트한다는 의미다)<br> 우리의 모델이 무엇을 forget해야하는지, 무엇을 remember해야 하는지 알았으면 좋겠다.<br> 그러니 새로운 것이 들어올 때, 모델은 더 이상 필요없다 판단한 어떤 long-term information을 첫번째로 잊는다. <br>그리고 새로운 input의 일부 중  사용할 가치가 있는 것을 배운다, 그리고 long-term memory에 저장한다.<br><br>

항상 full long-term memory를 사용하는 대신, 어떤 부분에 집중해야 하는지를 배운다.<br>
기본적으로, 우리는 forgetting, remembering, 그리고 attention을 위한 mechanisms이 필요하다. LSTM이 이 역할을 한다.<br><br>

vanilla RNN이 hidden state/memory 업데이트를 위해 하나의 equation을 사용하는 것에 비해:<br><br><br>
<img src='http://i.imgur.com/nT4VBPf.png' width='700' height='500'><br><br>

long term memory의 어떤 부분을 기억해야 하고 잊어야 할까?<br>
우리는 remember gate를 배우기 위해 새로운 input과 working memory를 사용할거다.<br>
new data의 어떤 부분을 사용해야 하고 저장해야 할까?<br>
attention vector를 사용해서 working memory를 업데이트 한다.<br><br>

- long-term memory는 대개 cell state로 불리운다.<br>
- working memory는 대개 hidden state로 불리운다. vanilla RNN의 hidden state와 유사하다.<br>
- remember vector는 대개 forget gate(forget gate의 1은 여전히 keep the memory를 의미하고, 0은 여전히 forget it을 의미함에도)로 불리운다.<br>
- save vector는 대개 input gate(얼마만큼의 input을 cell state에 넣을지를 결정한다.)로 불리운다.<br>
- focus vector는 대개 output gate로 불리운다<br><br><br>

------------

**활용 케이스**<br>

비디오<br>
[![IMAGE ALT TEXT HERE](http://img.youtube.com/vi/mLxsbWAYIpw/0.jpg)](https://www.youtube.com/watch?time_continue=14&v=mLxsbWAYIpw)<br><br>

현재로서 가장 유명한 application은 단어들이나 문장들, sound spectrogram 등등 sequential data와 관련있는 Natural Language Processing이다. <br>번역이나 sentiment analysis, 텍스트 생성 등등 에도 쓰인다.<br>

lstm의 application으로 덜 명확한 분야로는 이미지 분류(각 사진의 pixel을 행 by 행으로 feeding한다)나 딥마인드의 deep Q learning agents가 있다. <br><br>

**다른 끝내주는 예제들**<br>

텐서플로우로 음성 인식 (Speech recognition Tensorflow) - https://github.com/zzw922cn/Automatic_Speech_Recognition <br>
LSTM 비주얼라이제이션 (LSTM visualization) - https://github.com/HendrikStrobelt/LSTMVis<br><br>

**단계들**<br><br>

1. RNN 클래스 구축<br>
2. LSTM cell 클래스
구축<br>
3. 데이터 로딩 함수<br>
3. 트레이닝 시간!<br><br>

<img src='http://eric-yuan.me/wp-content/uploads/2015/06/5.jpg' width='300' height='300'>

In [ ]:
import numpy as np

class RecurrentNeuralNetwork:
    #input (word), expected output (next word), num of words (num of recurrences), array expected outputs, learning rate
    def __init__ (self, xs, ys, rl, eo, lr):
        #initial input (first word)
        self.x = np.zeros(xs)
        #input size 
        self.xs = xs
        #expected output (next word)
        self.y = np.zeros(ys)
        #output size
        self.ys = ys
        #weight matrix for interpreting results from LSTM cell (num words x num words matrix)
        self.w = np.random.random((ys, ys))
        #matrix used in RMSprop
        self.G = np.zeros_like(self.w)
        #length of the recurrent network - number of recurrences i.e num of words
        self.rl = rl
        #learning rate 
        self.lr = lr
        #array for storing inputs
        self.ia = np.zeros((rl+1,xs))
        #array for storing cell states
        self.ca = np.zeros((rl+1,ys))
        #array for storing outputs
        self.oa = np.zeros((rl+1,ys))
        #array for storing hidden states
        self.ha = np.zeros((rl+1,ys))
        #forget gate 
        self.af = np.zeros((rl+1,ys))
        #input gate
        self.ai = np.zeros((rl+1,ys))
        #cell state
        self.ac = np.zeros((rl+1,ys))
        #output gate
        self.ao = np.zeros((rl+1,ys))
        #array of expected output values
        self.eo = np.vstack((np.zeros(eo.shape[0]), eo.T))
        #declare LSTM cell (input, output, amount of recurrence, learning rate)
        self.LSTM = LSTM(xs, ys, rl, lr)
    
    #activation function. simple nonlinearity, convert nums into probabilities between 0 and 1
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    #the derivative of the sigmoid function. used to compute gradients for backpropagation
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))    
    
    #lets apply a series of matrix operations to our input (curr word) to compute a predicted output (next word)
    def forwardProp(self):
        for i in range(1, self.rl+1):
#             print(i,'아이임당')
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            #store computed cell state
            self.ca[i] = cs
            self.ha[i] = hs
            self.af[i] = f
            self.ai[i] = inp
            self.ac[i] = c
            self.ao[i] = o
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            self.x = self.eo[i-1]
        return self.oa
   
    
    def backProp(self):
        #update our weight matrices (Both in our Recurrent network, as well as the weight matrices inside LSTM cell)
        #init an empty error value 
        totalError = 0
        #initialize matrices for gradient updates
        #First, these are RNN level gradients
        #cell state
        dfcs = np.zeros(self.ys)
        #hidden state,
        dfhs = np.zeros(self.ys)
        #weight matrix
        tu = np.zeros((self.ys,self.ys))
        #Next, these are LSTM level gradients
        #forget gate
        tfu = np.zeros((self.ys, self.xs+self.ys))
        #input gate
        tiu = np.zeros((self.ys, self.xs+self.ys))
        #cell unit
        tcu = np.zeros((self.ys, self.xs+self.ys))
        #output gate
        tou = np.zeros((self.ys, self.xs+self.ys))
        #loop backwards through recurrences
        for i in range(self.rl, -1, -1):
            #error = calculatedOutput - expectedOutput
            error = self.oa[i] - self.eo[i]
            #calculate update for weight matrix
            #(error * derivative of the output) * hidden state
            tu += np.dot(np.atleast_2d(error * self.dsigmoid(self.oa[i])), np.atleast_2d(self.ha[i]).T)
            #Time to propagate error back to exit of LSTM cell
            #1. error * RNN weight matrix
            error = np.dot(error, self.w)
            #2. set input values of LSTM cell for recurrence i (horizontal stack of arrays, hidden + input)
            self.LSTM.x = np.hstack((self.ha[i-1], self.ia[i]))
            #3. set cell state of LSTM cell for recurrence i (pre-updates)
            self.LSTM.cs = self.ca[i]
            #Finally, call the LSTM cell's backprop, retreive gradient updates
            #gradient updates for forget, input, cell unit, and output gates + cell states & hiddens states
            fu, iu, cu, ou, dfcs, dfhs = self.LSTM.backProp(error, self.ca[i-1], self.af[i], self.ai[i], self.ac[i], self.ao[i], dfcs, dfhs)
            #calculate total error (not necesarry, used to measure training progress)
            totalError += np.sum(error)
            #accumulate all gradient updates
            #forget gate
            tfu += fu
            #input gate
            tiu += iu
            #cell state
            tcu += cu
            #output gate
            tou += ou
        #update LSTM matrices with average of accumulated gradient updates    
        self.LSTM.update(tfu/self.rl, tiu/self.rl, tcu/self.rl, tou/self.rl) 
        #update weight matrix with average of accumulated gradient updates  
        self.update(tu/self.rl)
        #return total error of this iteration
        return totalError
    
    def update(self, u):
        #vanilla implementation of RMSprop
        self.G = 0.9 * self.G + 0.1 * u**2  
        self.w -= self.lr/np.sqrt(self.G + 1e-8) * u
        return
    
    #this is where we generate some sample text after having fully trained our model
    #i.e error is below some threshold
    def sample(self):
        #loop through recurrences - start at 1 so the 0th entry of all arrays will be an array of 0's
        for i in range(1, self.rl+1):
            #set input for LSTM cell, combination of input (previous output) and previous hidden state
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            #run forward prop on the LSTM cell, retrieve cell state and hidden state
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            #store input as vector
            maxI = np.argmax(self.x)
            self.x = np.zeros_like(self.x)
            self.x[maxI] = 1
            self.ia[i] = self.x #Use np.argmax?
            #store cell states
            self.ca[i] = cs
            #store hidden state
            self.ha[i] = hs
            #forget gate
            self.af[i] = f
            #input gate
            self.ai[i] = inp
            #cell state
            self.ac[i] = c
            #output gate
            self.ao[i] = o
            #calculate output by multiplying hidden state with weight matrix
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            #compute new input
            maxI = np.argmax(self.oa[i])
            newX = np.zeros_like(self.x)
            newX[maxI] = 1
            self.x = newX
        #return all outputs    
        return self.oa

<img src='http://i.imgur.com/BUAVEZg.png' width='300' height='300'>

In [ ]:
class LSTM:
    # LSTM cell (input, output, amount of recurrence, learning rate)
    def __init__ (self, xs, ys, rl, lr):
        #input is word length x word length
        self.x = np.zeros(xs+ys)
        #input size is word length + word length
        self.xs = xs + ys
        #output 
        self.y = np.zeros(ys)
        #output size
        self.ys = ys
        #cell state intialized as size of prediction
        self.cs = np.zeros(ys)
        #how often to perform recurrence
        self.rl = rl
        #balance the rate of training (learning rate)
        self.lr = lr
        #init weight matrices for our gates
        #forget gate
        self.f = np.random.random((ys, xs+ys))
        #input gate
        self.i = np.random.random((ys, xs+ys))
        #cell state
        self.c = np.random.random((ys, xs+ys))
        #output gate
        self.o = np.random.random((ys, xs+ys))
        #forget gate gradient
        self.Gf = np.zeros_like(self.f)
        #input gate gradient
        self.Gi = np.zeros_like(self.i)
        #cell state gradient
        self.Gc = np.zeros_like(self.c)
        #output gate gradient
        self.Go = np.zeros_like(self.o)
    
    #activation function to activate our forward prop, just like in any type of neural network
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    #derivative of sigmoid to help computes gradients
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    #tanh! another activation function, often used in LSTM cells
    #Having stronger gradients: since data is centered around 0, 
    #the derivatives are higher. To see this, calculate the derivative 
    #of the tanh function and notice that input values are in the range [0,1].
    def tangent(self, x):
        return np.tanh(x)
    
    #derivative for computing gradients
    def dtangent(self, x):
        return 1 - np.tanh(x)**2
    
    #lets compute a series of matrix multiplications to convert our input into our output
    def forwardProp(self):
        f = self.sigmoid(np.dot(self.f, self.x))
        self.cs *= f
        i = self.sigmoid(np.dot(self.i, self.x))
        c = self.tangent(np.dot(self.c, self.x))
        self.cs += i * c
        o = self.sigmoid(np.dot(self.o, self.x))
        self.y = o * self.tangent(self.cs)
        return self.cs, self.y, f, i, c, o
    
   
    def backProp(self, e, pcs, f, i, c, o, dfcs, dfhs):
        #error = error + hidden state derivative. clip the value between -6 and 6.
        e = np.clip(e + dfhs, -6, 6)
        #multiply error by activated cell state to compute output derivative
        do = self.tangent(self.cs) * e
        #output update = (output deriv * activated output) * input
        ou = np.dot(np.atleast_2d(do * self.dtangent(o)).T, np.atleast_2d(self.x))
        #derivative of cell state = error * output * deriv of cell state + deriv cell
        dcs = np.clip(e * o * self.dtangent(self.cs) + dfcs, -6, 6)
        #deriv of cell = deriv cell state * input
        dc = dcs * i
        #cell update = deriv cell * activated cell * input
        cu = np.dot(np.atleast_2d(dc * self.dtangent(c)).T, np.atleast_2d(self.x))
        #deriv of input = deriv cell state * cell
        di = dcs * c
        #input update = (deriv input * activated input) * input
        iu = np.dot(np.atleast_2d(di * self.dsigmoid(i)).T, np.atleast_2d(self.x))
        #deriv forget = deriv cell state * all cell states
        df = dcs * pcs
        #forget update = (deriv forget * deriv forget) * input
        fu = np.dot(np.atleast_2d(df * self.dsigmoid(f)).T, np.atleast_2d(self.x))
        #deriv cell state = deriv cell state * forget
        dpcs = dcs * f
        #deriv hidden state = (deriv cell * cell) * output + deriv output * output * output deriv input * input * output + deriv forget
        #* forget * output
        dphs = np.dot(dc, self.c)[:self.ys] + np.dot(do, self.o)[:self.ys] + np.dot(di, self.i)[:self.ys] + np.dot(df, self.f)[:self.ys] 
        #return update gradinets for forget, input, cell, output, cell state, hidden state
        return fu, iu, cu, ou, dpcs, dphs
            
    def update(self, fu, iu, cu, ou):
        #update forget, input, cell, and output gradients
        self.Gf = 0.9 * self.Gf + 0.1 * fu**2 
        self.Gi = 0.9 * self.Gi + 0.1 * iu**2   
        self.Gc = 0.9 * self.Gc + 0.1 * cu**2   
        self.Go = 0.9 * self.Go + 0.1 * ou**2   
        
        #update our gates using our gradients
        self.f -= self.lr/np.sqrt(self.Gf + 1e-8) * fu
        self.i -= self.lr/np.sqrt(self.Gi + 1e-8) * iu
        self.c -= self.lr/np.sqrt(self.Gc + 1e-8) * cu
        self.o -= self.lr/np.sqrt(self.Go + 1e-8) * ou
        return

In [ ]:
def LoadText():
    #open text and return input and output data (series of words)
    with open("eminem_lose_yourself.txt", "r") as text_file:
        data = text_file.read()
    text = list(data)
    outputSize = len(text)
    data = list(set(text))
    uniqueWords, dataSize = len(data), len(data) 
    returnData = np.zeros((uniqueWords, dataSize))
    for i in range(0, dataSize):
        returnData[i][i] = 1
    returnData = np.append(returnData, np.atleast_2d(data), axis=0)
    output = np.zeros((uniqueWords, outputSize))
    for i in range(0, outputSize):
        index = np.where(np.asarray(data) == text[i])
        output[:,i] = returnData[0:-1,index[0]].astype(float).ravel()  
    return returnData, uniqueWords, output, outputSize, data

#write the predicted output (series of words) to disk
def ExportText(output, data):
    finalOutput = np.zeros_like(output)
    prob = np.zeros_like(output[0])
    outputText = ""
    print(len(data))
    print(output.shape[0])
    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            prob[j] = output[i][j] / np.sum(output[i])
        outputText += np.random.choice(data, p=prob)    
    with open("output.txt", "w") as text_file:
        text_file.write(outputText)
    return

In [ ]:
#Begin program    
print("Beginning")
iterations = 5000
learningRate = 0.001
#load input output data (words)
returnData, numCategories, expectedOutput, outputSize, data = LoadText()
print("Done Reading")
#init our RNN using our hyperparams and dataset
RNN = RecurrentNeuralNetwork(numCategories, numCategories, outputSize, expectedOutput, learningRate)

#training time!
for i in range(1, iterations, 100):
    #compute predicted next word
    RNN.forwardProp()
    #update all our weights using our error
    error = RNN.backProp()
    #once our error/loss is small enough
    print("Error on iteration ", i, ": ", error)
    if error > -100 and error < 100 or i % 100 == 0:
        #we can finally define a seed word
        seed = np.zeros_like(RNN.x)
        maxI = np.argmax(np.random.random(RNN.x.shape))
        seed[maxI] = 1
        RNN.x = seed  
        #and predict some new text!
        output = RNN.sample()
        print(output)    
        #write it all to disk
        ExportText(output, data)
        print("Done Writing")
print("Complete")